## Answer questions

In [171]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [172]:
import pandas as pd
from src.paths import TRANSFORMED_DATA_DIR, MODELS_DIR, RAW_DATA_DIR

In [173]:
data = pd.read_csv(TRANSFORMED_DATA_DIR / 'transformed_data.csv')

In [174]:
data.shape

(4817, 22)

In [175]:
# Convert date columns to datetime
data['registration_date'] = pd.to_datetime(data['registration_date'])
data['sold_at'] = pd.to_datetime(data['sold_at'])

In [176]:
# Change data types from object to categorical
from src.data import convert_object_columns_to_category

data = convert_object_columns_to_category(data)

In [177]:
from src.data import get_train_test_data

In [178]:
import joblib

In [179]:
features = joblib.load(MODELS_DIR / 'features.pkl')
target = joblib.load(MODELS_DIR / 'target.pkl')

In [180]:
X, y, X_train, X_test, y_train, y_test = get_train_test_data(data, features, target)

In [181]:
# Load the model
model = joblib.load(MODELS_DIR / 'model.pkl')

### Q1

In [182]:
# Read feature importance df from models folder
feature_importance = joblib.load(MODELS_DIR / 'feature_importance_df.pkl')

In [183]:
feature_importance

,feature,importance
17,model_initial,0.248121
0,model_key,0.176764
13,feature_8,0.110575
14,age_in_months_when_sold,0.091855
12,feature_7,0.085964
2,engine_power,0.048129
8,feature_3,0.035199
10,feature_5,0.033560
1,mileage,0.032679
7,feature_2,0.029472


### Q2

As found during the data exploration phase:
- Hybrid and electrical cars are more expensive on average.
- Electrical cars average prices were stable from winter to summer, and were not sold in autumn.
- Diesel and petrol cars had similar average prices, although petrol cars had a drop in average prices starting summer 2018.
- The most expensive car type is, on average, suv, although coupe was most expensive at the start of the year and then dropped below suv also starting in summer.
- Coupe and convertible cars were, on average, more expensive in winter than in summer.
- Vans where more expensive, on average, in spring, summer, and autumn, than in winter.
- Subcompact had generally the lowest average prices.
- Paint color does not seem to generally determine or be associated with the average price, except for color green, which consistently had prices much lower than other colors. Maybe not very popular.
- Orange and white cars were sold for more, on average, during summer than during winter and spring.
- Red cars were the opposite, with lower average prices during summer than during winter and spring.

Find similar observations using estimated price instead of real price

In [184]:
pred = model.predict(X)

In [185]:
data_q2 = data.copy()

In [186]:
data_q2['price'] = pred

In [187]:
from src.plots import plot_avg_target_time_series_by_features

In [188]:
# Load car features
car_features = joblib.load(RAW_DATA_DIR / 'car_features.pkl')

In [189]:
plot_avg_target_time_series_by_features(data_q2, car_features)

In [190]:
# Load small cardinality features
small_cardinality_features = joblib.load(RAW_DATA_DIR / 'small_cardinality_features.pkl')

In [191]:
plot_avg_target_time_series_by_features(data_q2, small_cardinality_features)

In [192]:
from src.questions import ttest_mean_price_difference_between_groups_after_filter

In [193]:
grouping_column = 'season_sold_at'
group_1 = 'winter'
group_2 = 'summer'
for feature in small_cardinality_features:
    for feature_value in data_q2[feature].unique():
        t_stat, p_val = ttest_mean_price_difference_between_groups_after_filter(data_q2, feature, feature_value, grouping_column, group_1, group_2)
        print(f'Feature: {feature}, Feature Value: {feature_value}, T-stat: {t_stat:.4f}, P-value: {p_val:.4f}') 

Feature: fuel, Feature Value: diesel, T-stat: -1.0547, P-value: 0.2917
Feature: fuel, Feature Value: petrol, T-stat: 1.7016, P-value: 0.0931
Feature: fuel, Feature Value: hybrid_petrol, T-stat: nan, P-value: nan
Feature: fuel, Feature Value: electro, T-stat: 0.4836, P-value: 0.7132
Feature: car_type, Feature Value: convertible, T-stat: 0.9885, P-value: 0.3347
Feature: car_type, Feature Value: coupe, T-stat: 3.6123, P-value: 0.0008
Feature: car_type, Feature Value: estate, T-stat: 2.1425, P-value: 0.0325
Feature: car_type, Feature Value: hatchback, T-stat: -0.8657, P-value: 0.3873
Feature: car_type, Feature Value: sedan, T-stat: 3.9536, P-value: 0.0001
Feature: car_type, Feature Value: subcompact, T-stat: 0.2625, P-value: 0.7942
Feature: car_type, Feature Value: suv, T-stat: -1.2127, P-value: 0.2257
Feature: car_type, Feature Value: van, T-stat: -3.4716, P-value: 0.0029
Feature: paint_color, Feature Value: black, T-stat: 1.0008, P-value: 0.3172
Feature: paint_color, Feature Value: grey,

### Q3

In [194]:
today_date = '3/1/2024'

In [195]:
data_q3 = data.copy()

In [196]:
# Convert date columns to datetime and mock the sold_at date as today's date
data_q3['registration_date'] = pd.to_datetime(data_q3['registration_date'])
data_q3['sold_at'] = today_date
data_q3['sold_at'] = pd.to_datetime(data_q3['sold_at'])

In [197]:
# Calculate mileage per month
data_q3['mileage_per_month'] = data_q3['mileage'] / data_q3['age_in_months_when_sold']

In [198]:
# Calculate age in month at today date and replace in data_q3
data_q3['age_in_months_when_sold'] = (data_q3['sold_at'].dt.to_period('M') - data_q3['registration_date'].dt.to_period('M')).apply(lambda x: x.n)

In [199]:
# Update estimated mileage at today date
data_q3['mileage'] = data_q3['age_in_months_when_sold'] * data_q3['mileage_per_month']

In [200]:
data_q3[['registration_date', 'sold_at', 'age_in_months_when_sold', 'mileage']].head()

,registration_date,sold_at,age_in_months_when_sold,mileage
0,2012-02-01,2024-03-01,145,286754.859155
1,2016-04-01,2024-03-01,95,60147.954545
2,2012-04-01,2024-03-01,143,374449.585714
3,2014-07-01,2024-03-01,116,345396.744186
4,2014-12-01,2024-03-01,111,269444.175000


In [201]:
X_q3, y_q3, X_train_q3, X_test_q3, y_train_q3, y_test_q3 = get_train_test_data(data_q3, features, target)

In [202]:
# Get estimated prices today
pred_q3 = model.predict(X_q3)

In [203]:
# Add 1 year to the age_in_months_when_sold
X_q3['age_in_months_when_sold'] = X_q3['age_in_months_when_sold'] + 12

In [204]:
# Add 1 year worth of mileage
X_q3['mileage'] = X_q3['mileage'] + 12 * data_q3['mileage_per_month']

In [205]:
# Get estimated prices 1 year later
pred_q3_one_year_later = model.predict(X_q3)

In [206]:
X_q3['price_today'] = pred_q3
X_q3['price_one_year_later'] = pred_q3_one_year_later
X_q3['loss'] = X_q3['price_today'] - X_q3['price_one_year_later'] 

In [207]:
# Indentify cars that are candidates for buying
candidate_cars = X_q3[(X_q3['price_today'] >= 20000) & (X_q3['loss'] <= 1000)]

In [208]:
# Show top 10 candidate cars with lowest loss
candidate_cars.sort_values('loss').head(10)

,model_key,mileage,engine_power,fuel,paint_color,car_type,feature_1,feature_2,feature_3,feature_4,...,feature_6,feature_7,feature_8,age_in_months_when_sold,month_sold_at,season_sold_at,model_initial,price_today,price_one_year_later,loss
4615,X6,103518.040816,180,diesel,black,suv,True,True,False,True,...,False,True,True,128,8,summer,X,23899.460938,28647.587891,-4748.126953
4126,X5,85108.604651,230,diesel,silver,suv,False,False,False,False,...,False,False,True,126,4,spring,X,21182.710938,22385.962891,-1203.251953
4210,X6,224153.090909,230,diesel,black,suv,True,True,True,True,...,True,True,True,104,5,spring,X,20349.667969,21180.527344,-830.859375
4723,X6,102189.375000,280,diesel,white,suv,True,True,True,True,...,True,True,True,111,8,summer,X,24080.552734,24630.833984,-550.281250
4717,X6,165463.027027,230,diesel,grey,suv,True,True,False,True,...,False,True,True,116,8,summer,X,21289.566406,21748.347656,-458.781250
4106,X6,169312.064516,230,diesel,grey,suv,True,True,True,False,...,False,True,True,114,4,spring,X,21002.169922,21424.222656,-422.052734
3073,M3,146533.333333,317,petrol,black,sedan,True,True,False,False,...,True,True,True,112,5,spring,M,23852.378906,23703.550781,148.828125
4658,X4,273108.208333,140,diesel,grey,suv,True,True,False,False,...,False,True,True,127,8,summer,X,29820.486328,29619.423828,201.062500
67,M3,66996.268657,309,petrol,silver,coupe,True,True,False,False,...,True,True,True,150,4,spring,M,28174.792969,27918.562500,256.230469
3910,X5,9278.857143,155,diesel,black,suv,True,True,False,True,...,True,True,True,92,2,winter,X,23939.302734,23550.667969,388.634766


In [209]:
# Find the value counts for model_key in candidate_cars
candidate_cars['model_key'].value_counts()

model_key
X6                  5
X5                  2
M3                  2
740                 1
640 Gran Coupé      1
                   ..
325 Gran Turismo    0
325                 0
320 Gran Turismo    0
320                 0
i8                  0
Name: count, Length: 75, dtype: int64

In [210]:
# Describe numerical features of candidate cars
candidate_cars.describe()

,mileage,engine_power,age_in_months_when_sold,month_sold_at,price_today,price_one_year_later,loss
count,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000
mean,134012.201568,245.307692,116.692308,5.384615,23801.533203,24155.386719,-353.853058
std,79411.463712,74.589303,14.733095,1.980676,3395.450195,3451.513428,1481.070679
min,9278.857143,140.000000,92.000000,2.000000,20349.667969,20082.744141,-4748.126953
25%,85108.604651,230.000000,107.000000,4.000000,21117.591797,21424.222656,-550.281250
50%,146533.333333,230.000000,114.000000,5.000000,23852.378906,23550.667969,148.828125
75%,181614.681818,280.000000,126.000000,8.000000,24080.552734,27918.562500,388.634766
max,273108.208333,423.000000,150.000000,8.000000,29820.486328,29619.423828,917.826172


In [211]:
# Show the car with the lowest loss
car_index = candidate_cars.sort_values('loss').index[0]
data_q3.loc[car_index]

maker_key                                  BMW
model_key                                   X6
mileage                            93813.22449
engine_power                               180
registration_date          2014-07-01 00:00:00
fuel                                    diesel
paint_color                              black
car_type                                   suv
feature_1                                 True
feature_2                                 True
feature_3                                False
feature_4                                 True
feature_5                                 True
feature_6                                False
feature_7                                 True
feature_8                                 True
price                                    43600
sold_at                    2024-03-01 00:00:00
age_in_months_when_sold                    116
month_sold_at                                8
season_sold_at                          summer
model_initial

In [212]:
data.loc[car_index]

maker_key                                  BMW
model_key                                   X6
mileage                                  39628
engine_power                               180
registration_date          2014-07-01 00:00:00
fuel                                    diesel
paint_color                              black
car_type                                   suv
feature_1                                 True
feature_2                                 True
feature_3                                False
feature_4                                 True
feature_5                                 True
feature_6                                False
feature_7                                 True
feature_8                                 True
price                                    43600
sold_at                    2018-08-01 00:00:00
age_in_months_when_sold                     49
month_sold_at                                8
season_sold_at                          summer
model_initial

In [236]:
X_q3.loc[car_index]

model_key                            X3
mileage                    72054.818182
engine_power                        135
fuel                             diesel
paint_color                      silver
car_type                            suv
feature_1                         False
feature_2                         False
feature_3                         False
feature_4                         False
feature_5                         False
feature_6                         False
feature_7                         False
feature_8                          True
age_in_months_when_sold              69
month_sold_at                         7
season_sold_at                   summer
model_initial                         X
price_today                26070.072266
price_one_year_later       48791.386719
loss                      -22721.314453
Name: 3763, dtype: object

### Q3 assumming today is a month after latest sold at date in the data

In [215]:
today_date = data['sold_at'].max()

In [217]:
data_q3 = data.copy()

In [218]:
# Convert date columns to datetime and mock the sold_at date as today's date
data_q3['registration_date'] = pd.to_datetime(data_q3['registration_date'])
data_q3['sold_at'] = today_date
data_q3['sold_at'] = pd.to_datetime(data_q3['sold_at'])

In [219]:
# Calculate mileage per month
data_q3['mileage_per_month'] = data_q3['mileage'] / data_q3['age_in_months_when_sold']

In [220]:
# Calculate age in month at today date and replace in data_q3
data_q3['age_in_months_when_sold'] = (data_q3['sold_at'].dt.to_period('M') - data_q3['registration_date'].dt.to_period('M')).apply(lambda x: x.n)

In [221]:
# Update estimated mileage at today date
data_q3['mileage'] = data_q3['age_in_months_when_sold'] * data_q3['mileage_per_month']

In [222]:
data_q3[['registration_date', 'sold_at', 'age_in_months_when_sold', 'mileage']].head()

,registration_date,sold_at,age_in_months_when_sold,mileage
0,2012-02-01,2018-09-01,79,156231.957746
1,2016-04-01,2018-09-01,29,18360.954545
2,2012-04-01,2018-09-01,77,201626.700000
3,2014-07-01,2018-09-01,50,148877.906977
4,2014-12-01,2018-09-01,45,109234.125000


In [223]:
X_q3, y_q3, X_train_q3, X_test_q3, y_train_q3, y_test_q3 = get_train_test_data(data_q3, features, target)

In [224]:
# Get estimated prices today
pred_q3 = model.predict(X_q3)

In [225]:
# Add 1 year to the age_in_months_when_sold
X_q3['age_in_months_when_sold'] = X_q3['age_in_months_when_sold'] + 12

In [226]:
# Add 1 year worth of mileage
X_q3['mileage'] = X_q3['mileage'] + 12 * data_q3['mileage_per_month']

In [227]:
# Get estimated prices 1 year later
pred_q3_one_year_later = model.predict(X_q3)

In [228]:
X_q3['price_today'] = pred_q3
X_q3['price_one_year_later'] = pred_q3_one_year_later
X_q3['loss'] = X_q3['price_today'] - X_q3['price_one_year_later'] 

In [229]:
# Indentify cars that are candidates for buying
candidate_cars = X_q3[(X_q3['price_today'] >= 20000) & (X_q3['loss'] <= 1000)]

In [230]:
# Show top 10 candidate cars with lowest loss
candidate_cars.sort_values('loss').head(10)

,model_key,mileage,engine_power,fuel,paint_color,car_type,feature_1,feature_2,feature_3,feature_4,...,feature_6,feature_7,feature_8,age_in_months_when_sold,month_sold_at,season_sold_at,model_initial,price_today,price_one_year_later,loss
3763,X3,72054.818182,135,diesel,silver,suv,False,False,False,False,...,False,False,True,69,7,summer,X,26070.072266,48791.386719,-22721.314453
2125,520 Gran Turismo,71985.413793,135,diesel,brown,hatchback,True,True,False,False,...,False,True,True,74,5,spring,5,20635.966797,43051.722656,-22415.755859
4748,X4,106621.351351,140,diesel,silver,suv,True,True,False,False,...,False,True,True,49,9,autumn,X,48293.261719,68922.750000,-20629.488281
4731,X4,78519.400000,140,diesel,grey,suv,False,False,False,False,...,False,False,True,57,9,autumn,X,28752.408203,48940.933594,-20188.525391
3845,X3,36713.695652,190,diesel,red,suv,False,False,False,True,...,True,True,True,65,2,winter,X,25745.140625,42903.843750,-17158.703125
2924,520,57133.645833,135,diesel,grey,sedan,True,True,False,False,...,False,True,True,65,4,spring,5,20849.996094,37219.753906,-16369.757812
4618,X4,72836.052632,140,diesel,blue,suv,True,False,False,True,...,False,True,True,51,8,summer,X,30520.873047,43531.406250,-13010.533203
3765,X6,103089.681818,180,diesel,white,suv,False,False,False,False,...,False,False,True,81,6,summer,X,29053.320312,41801.531250,-12748.210938
4117,X4,140770.341463,140,diesel,grey,suv,True,True,False,False,...,False,True,True,56,6,summer,X,28090.244141,38582.242188,-10491.998047
4153,X4,83456.000000,140,diesel,white,suv,True,True,False,True,...,False,True,True,52,8,summer,X,33269.035156,43322.214844,-10053.179688


In [231]:
# Find the value counts for model_key in candidate_cars
candidate_cars['model_key'].value_counts()

model_key
X4                29
X3                19
420 Gran Coupé    14
420               14
X5                 8
                  ..
425                0
430                0
116                0
518                0
i8                 0
Name: count, Length: 75, dtype: int64

In [232]:
# Describe numerical features of candidate cars
candidate_cars.describe()

,mileage,engine_power,age_in_months_when_sold,month_sold_at,price_today,price_one_year_later,loss
count,151.000000,151.000000,151.000000,151.000000,151.000000,151.000000,151.000000
mean,113415.463444,161.900662,60.655629,5.139073,26455.765625,28335.667969,-1879.902710
std,64415.733388,42.304413,16.483141,2.209192,6107.244141,8382.097656,4600.388672
min,2622.285714,75.000000,26.000000,1.000000,20072.343750,19414.066406,-22721.314453
25%,71448.956844,135.000000,51.000000,3.000000,21922.107422,21970.511719,-1821.472656
50%,94747.692308,140.000000,57.000000,5.000000,24049.505859,25422.476562,-220.898438
75%,149193.153245,190.000000,66.500000,7.000000,29483.652344,33028.363281,494.853516
max,343224.573171,317.000000,125.000000,9.000000,48293.261719,68922.750000,989.701172


In [233]:
# Show the car with the lowest loss
car_index = candidate_cars.sort_values('loss').index[0]
data_q3.loc[car_index]

maker_key                                  BMW
model_key                                   X3
mileage                           59523.545455
engine_power                               135
registration_date          2013-12-01 00:00:00
fuel                                    diesel
paint_color                             silver
car_type                                   suv
feature_1                                False
feature_2                                False
feature_3                                False
feature_4                                False
feature_5                                False
feature_6                                False
feature_7                                False
feature_8                                 True
price                                    26700
sold_at                    2018-09-01 00:00:00
age_in_months_when_sold                     57
month_sold_at                                7
season_sold_at                          summer
model_initial

In [234]:
data.loc[car_index]

maker_key                                  BMW
model_key                                   X3
mileage                                  57435
engine_power                               135
registration_date          2013-12-01 00:00:00
fuel                                    diesel
paint_color                             silver
car_type                                   suv
feature_1                                False
feature_2                                False
feature_3                                False
feature_4                                False
feature_5                                False
feature_6                                False
feature_7                                False
feature_8                                 True
price                                    26700
sold_at                    2018-07-01 00:00:00
age_in_months_when_sold                     55
month_sold_at                                7
season_sold_at                          summer
model_initial

In [235]:
X_q3.loc[car_index]

model_key                            X3
mileage                    72054.818182
engine_power                        135
fuel                             diesel
paint_color                      silver
car_type                            suv
feature_1                         False
feature_2                         False
feature_3                         False
feature_4                         False
feature_5                         False
feature_6                         False
feature_7                         False
feature_8                          True
age_in_months_when_sold              69
month_sold_at                         7
season_sold_at                   summer
model_initial                         X
price_today                26070.072266
price_one_year_later       48791.386719
loss                      -22721.314453
Name: 3763, dtype: object

### Q4

Check training notebook

### Q5

Add other findings from data exploration